In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import statsmodels.api as sm
from scipy import stats
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn.model_selection import cross_val_score

In [2]:
crop_data=pd.read_csv(r"C:\Users\navsi\Downloads\crop_yield.csv.zip")

In [3]:
crop_data

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909
...,...,...,...,...,...,...,...,...,...,...
19684,Small millets,1998,Kharif,Nagaland,4000.0,2000,1498.0,395200.00,1160.00,0.500000
19685,Wheat,1998,Rabi,Nagaland,1000.0,3000,1498.0,98800.00,290.00,3.000000
19686,Maize,1997,Kharif,Jammu and Kashmir,310883.0,440900,1356.2,29586735.11,96373.73,1.285000
19687,Rice,1997,Kharif,Jammu and Kashmir,275746.0,5488,1356.2,26242746.82,85481.26,0.016667


In [4]:
scaler = MinMaxScaler()
crop_data['Area'] = scaler.fit_transform(crop_data[['Area']])
crop_data['Production'] = scaler.fit_transform(crop_data[['Production']])
crop_data['Annual_Rainfall'] = scaler.fit_transform(crop_data[['Annual_Rainfall']])
crop_data['Fertilizer'] = scaler.fit_transform(crop_data[['Fertilizer']])
crop_data['Pesticide'] = scaler.fit_transform(crop_data[['Pesticide']])
crop_data['Yield'] = scaler.fit_transform(crop_data[['Yield']])

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
crop_data['Area'] = scaler.fit_transform(crop_data[['Area']])
crop_data['Production'] = scaler.fit_transform(crop_data[['Production']])
crop_data['Annual_Rainfall'] = scaler.fit_transform(crop_data[['Annual_Rainfall']])
crop_data['Fertilizer'] = scaler.fit_transform(crop_data[['Fertilizer']])
crop_data['Pesticide'] = scaler.fit_transform(crop_data[['Pesticide']])
crop_data['Yield'] = scaler.fit_transform(crop_data[['Yield']])

In [6]:
crop_data.head()

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,-0.144802,-0.062267,0.751197,-0.179880,-0.121745,-0.090128
1,Arhar/Tur,1997,Kharif,Assam,-0.236473,-0.062464,0.751197,-0.247217,-0.219385,-0.090225
2,Castor seed,1997,Kharif,Assam,-0.244443,-0.062482,0.751197,-0.253072,-0.227875,-0.090763
3,Coconut,1997,Whole Year,Assam,-0.218707,0.419954,0.751197,-0.234167,-0.200462,5.872928
4,Cotton(lint),1997,Kharif,Assam,-0.243157,-0.062479,0.751197,-0.252127,-0.226504,-0.090555


In [7]:
# Selecting specific columns for outlier detection
selected_columns = ['Area', 'Production', 'Annual_Rainfall', 'Fertilizer', 'Pesticide', 'Yield']

# Function to find and remove outliers using the IQR method for selected columns
def remove_outliers(data, columns):
    for column in columns:
        # Calculate the first quartile (Q1) and third quartile (Q3)
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        
        # Calculate the IQR (Interquartile Range)
        IQR = Q3 - Q1
        
        # Define the lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Find and remove outliers
        outliers_mask = (data[column] < lower_bound) | (data[column] > upper_bound)
        data = data[~outliers_mask]
    
    return data

# Call the function with your DataFrame and selected columns
crop_data_no_outliers = remove_outliers(crop_data, selected_columns)

# Display the shape of the DataFrame before and after removing outliers
print("Shape of the original DataFrame:", crop_data.shape)
print("Shape of the DataFrame after removing outliers:", crop_data_no_outliers.shape)


Shape of the original DataFrame: (19689, 10)
Shape of the DataFrame after removing outliers: (8779, 10)


In [8]:
crop_data_no_outliers

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
1,Arhar/Tur,1997,Kharif,Assam,-0.236473,-0.062464,0.751197,-0.247217,-0.219385,-0.090225
2,Castor seed,1997,Kharif,Assam,-0.244443,-0.062482,0.751197,-0.253072,-0.227875,-0.090763
4,Cotton(lint),1997,Kharif,Assam,-0.243157,-0.062479,0.751197,-0.252127,-0.226504,-0.090555
5,Dry chillies,1997,Whole Year,Assam,-0.226989,-0.062448,0.751197,-0.240250,-0.209283,-0.090302
6,Gram,1997,Rabi,Assam,-0.241464,-0.062476,0.751197,-0.250884,-0.224702,-0.090504
...,...,...,...,...,...,...,...,...,...,...
19677,Urad,2018,Autumn,Odisha,-0.226807,-0.062469,0.242560,-0.230431,-0.206517,-0.090651
19678,Urad,2018,Summer,Odisha,-0.239292,-0.062473,0.242560,-0.246061,-0.221531,-0.090500
19680,Wheat,2018,Summer,Odisha,-0.245329,-0.062481,0.242560,-0.253619,-0.228791,-0.088956
19684,Small millets,1998,Kharif,Nagaland,-0.240071,-0.062475,0.073749,-0.249707,-0.223593,-0.090465


In [9]:
# Features (X) are all numeric columns except 'Yield', which is the target variable (label)
numeric_features = crop_data_no_outliers.select_dtypes(include=np.number).columns
X = crop_data_no_outliers[numeric_features]
y = crop_data_no_outliers['Yield']

# Select the last 5 numeric features
last_5_features = X.columns[-5:]
X_selected = X[last_5_features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)

# Regression Model
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)
y_pred_regression = linear_reg_model.predict(X_test)
mse_regression = mean_squared_error(y_test, y_pred_regression)
print(f"Mean Squared Error (Linear Regression): {mse_regression}")


Mean Squared Error (Linear Regression): 1.925929944387236e-34


In [10]:
from sklearn.tree import DecisionTreeRegressor

# Decision Tree Regression Model
decision_tree_model = DecisionTreeRegressor(random_state=0)
decision_tree_model.fit(X_train, y_train)
y_pred_decision_tree = decision_tree_model.predict(X_test)

# Evaluate the Decision Tree model
mse_decision_tree = mean_squared_error(y_test, y_pred_decision_tree)
print(f"Mean Squared Error (Decision Tree Regression): {mse_decision_tree}")



Mean Squared Error (Decision Tree Regression): 3.07260734451717e-12


In [11]:
# XGBoost Regression Model
xgboost_model = xgb.XGBRegressor(random_state=0)
xgboost_model.fit(X_train, y_train)
y_pred_xgboost = xgboost_model.predict(X_test)

# Evaluate the XGBoost model
mse_xgboost = mean_squared_error(y_test, y_pred_xgboost)
print(f"Mean Squared Error (XGBoost Regression): {mse_xgboost}")



Mean Squared Error (XGBoost Regression): 7.139316693887611e-10


In [12]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest Regression Model
random_forest_model = RandomForestRegressor(random_state=0)
random_forest_model.fit(X_train, y_train)
y_pred_random_forest = random_forest_model.predict(X_test)

# Evaluate the Random Forest model
mse_random_forest = mean_squared_error(y_test, y_pred_random_forest)
print(f"Mean Squared Error (Random Forest Regression): {mse_random_forest}")




Mean Squared Error (Random Forest Regression): 1.6513100640850584e-12


In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Create a categorical target variable based on Yield
crop_data['Yield_Class'] = pd.cut(crop_data['Yield'], bins=[-np.inf, 0.25, 0.75, np.inf], labels=['Low', 'Medium', 'High'])

# Features (X) are all numeric columns except 'Yield', which is the original target variable (label)
numeric_features_classification = crop_data.select_dtypes(include=np.number).columns
X_classification = crop_data[numeric_features_classification]
y_classification = crop_data['Yield_Class']

# Split the data into training and testing sets
X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(X_classification, y_classification, test_size=0.2, random_state=0)

# Classification Model - Decision Tree
decision_tree_classifier = DecisionTreeClassifier(random_state=0)
decision_tree_classifier.fit(X_train_classification, y_train_classification)
y_pred_decision_tree_classification = decision_tree_classifier.predict(X_test_classification)

# Evaluate the Decision Tree Classifier
accuracy_decision_tree = accuracy_score(y_test_classification, y_pred_decision_tree_classification)
confusion_matrix_decision_tree = confusion_matrix(y_test_classification, y_pred_decision_tree_classification)

print(f"Accuracy (Decision Tree Classifier): {accuracy_decision_tree}")
print("Confusion Matrix (Decision Tree Classifier):\n", confusion_matrix_decision_tree)

Accuracy (Decision Tree Classifier): 0.9992381919756221
Confusion Matrix (Decision Tree Classifier):
 [[  30    0    1]
 [   0 3904    2]
 [   0    0    1]]
